# Mine releases from selected projects

This notebook mine the releases

In [1]:
import os
import datetime
import json
import re
import traceback
import math

from multiprocessing import Pool
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

import releasy

In [2]:
from util import (
    DATA_PATH,
    REPO_PATH,
    TMP_PATH, 
    CPU,
    CycleType,
    RAPID_RELEASE_LIM,
    TRAD_RELEASE_LIM,
    delta2days,
    is_rapid_release,
    is_trad_release,
)

In [3]:
selected_projects = pd.read_csv(DATA_PATH / '10_projects_selected.csv')

## Mine Releases


In [4]:
def mine(name: str) -> releasy.Project:
    elapsed_time = datetime.datetime.now()
    
    try:
        repo_path = str(REPO_PATH / name)
        project = releasy.Miner(repo_path, name).apply(
            releasy.FinalReleaseMiner(),
            releasy.HistoryCommitMiner(),
            releasy.BaseReleaseMiner(),
            releasy.ContributorMiner(),
            releasy.SemanticReleaseMiner()
        ).mine()

        rapid_releases = [release for release in project.main_releases if is_rapid_release(release)]
        trad_releases = [release for release in project.main_releases if is_trad_release(release)]

        project_data = {
            'project': project.name,
            'prefixes': len(project.releases.prefixes()),
            'prefixes_names': str(" ".join(project.releases.prefixes())),
            'main_releases': len(project.main_releases),
            'rapid_releases': len(rapid_releases),
            'trad_releases': len(trad_releases),
            'patches': len(project.patches),
            'releases': len(project.releases)
        }
        
        release_data = []
        for srelease in project.main_releases:
            # release = srelease.release
            cycle_days = delta2days(srelease.cycle)
            delay = delta2days(srelease.delay)
            prev_release = srelease.prev_main_release

            if srelease.commits:
                duration = delta2days(srelease.time - srelease.commits.first(lambda c: c.author_time).author_time)
            else:
                duration = 0
                
            if srelease.commits and prev_release:
                commits_in_advance = [commit for commit in srelease.commits if commit.author_time < prev_release.time]
            else: 
                commits_in_advance = []
                
            
            release_data.append({
                'project': project.name,
                'release': srelease.name,
                'previous_release': srelease.prev_main_release.name if srelease.prev_main_release else '',
                'cycle': srelease.cycle,
                'cycle_days': cycle_days,
                'rapid_release': is_rapid_release(srelease),
                'trad_release': is_trad_release(srelease),
                
                'commits': len(srelease.commits),
                'commits_in_advance': len(commits_in_advance),
                'start_delay': delay,
                'duration': duration,
                'patches': len(srelease.patches)     
            })
        
        elapsed_time = datetime.datetime.now() - elapsed_time
        project_data['time'] = elapsed_time
    except Exception as err:
        print(f"{name:40} {err=}")
        traceback.print_exception(err)
        project_data = {}
        release_data = []
    
    return project_data, release_data

In [5]:
project_names = list(selected_projects['project'])
# project_names = project_names[:1]

In [6]:
with Pool(processes=CPU) as pool:
    processed = 0
    project_results = []
    release_results = []
    progress = IntProgress(min=0, max=len(project_names))
    display(progress)
    for result in pool.imap_unordered(mine, project_names):
        project_result, release_result = result
        project_results.append(project_result)
        release_results.extend(release_result)
        progress.value += 1


IntProgress(value=0, max=1665)

In [7]:
projects = pd.DataFrame(project_results)
projects.sample(1)

,project,prefixes,prefixes_names,main_releases,rapid_releases,trad_releases,patches,releases,time
1039,baidu/amis,1,,9,5,2,37,46,0 days 00:00:00.414619


In [8]:
releases = pd.DataFrame(release_results)
releases.sample(10)

,project,release,previous_release,cycle,cycle_days,rapid_release,trad_release,commits,commits_in_advance,start_delay,duration,patches
17934,hashicorp/terraform-provider-aws,v2.35.0,v2.34.0,7 days 02:07:12,7.088333,True,False,120,20,-23.927211,31.015544,0
20467,electron/fiddle,v0.27.0,v0.26.0,109 days 04:05:21,109.170382,False,True,59,0,5.159352,104.011030,3
14231,wix/react-native-calendars,1.394.0,1.393.0,0 days 00:13:18,0.009236,False,False,0,0,0.000000,0.000000,0
1875,kaorun343/vue-property-decorator,v8.3.0,v8.2.0,51 days 03:36:18,51.150208,False,False,9,0,28.700509,22.449699,3
33793,netbox-community/netbox,v2.4.0,v2.3.0,160 days 21:16:57,160.886771,False,True,193,0,31.802998,129.083773,9
41345,gchq/CyberChef,v7.4.0,v7.3.0,0 days 21:43:18,0.905069,True,False,7,0,0.066701,0.838368,0
3865,microsoft/Windows-driver-samples,85524,85523,0 days 00:00:00,0.000000,False,False,0,0,0.000000,0.000000,0
49240,airbnb/lottie-android,v2.5.0,v2.3.0,94 days 22:25:30,94.934375,False,True,75,0,19.919502,75.014873,6
48723,grafana/grafana,v1.7.0,v1.6.0,55 days 22:37:24,55.942639,False,False,190,20,-8.706551,64.649190,0
49464,rclone/rclone,v1.30,v1.29,60 days 22:11:17,60.924502,False,False,72,2,-20.457072,81.381574,0


In [9]:
len(projects)

1665

In [10]:
len(releases)

52836

In [11]:
len(releases.query("rapid_release == True"))

29306

In [12]:
len(releases.query("trad_release == True"))

11296

In [13]:
len(releases.query("rapid_release == False and trad_release == False"))

12234

In [14]:
releases['commits'].sum()

5520217

In [15]:
projects.to_csv(DATA_PATH / '20_projects.csv', index=False)

In [16]:
releases.to_csv(DATA_PATH / '20_releases.csv', index=False)